# Air Quality Analysis - San Francisco PM 2.5 May
1. Retrieve small particle data for San Francisco in May 2018,2019, & 2020.
2. Explore and compare data retreived from API vs. S3 storage.
3. Prepare data to be merged with existing Jan-Apr data.

## Gathering Data (Options and Information)
Options for gathering data:
1. OpenAQ API
 - Access the most recent 90 days of data
 - Accessible via a URL Query
 - Documentation - https://docs.openaq.org/
2. OpenAQ AWS S3 Retention
 - Access full dataset
 - Accessible via query tool such as Amazon Athena, Apache Spark, or Google BigQuery
 - Documentation - https://gist.github.com/jflasher/573525aff9a5d8a966e5718272ceb25a
 - Medium Tutorial - https://medium.com/@openaq/how-in-the-world-do-you-access-air-quality-data-older-than-90-days-on-the-openaq-platform-8562df519ecd
3. EPA AirNow API
 - Documentation - https://docs.airnowapi.org/

### OpenAQ API (URL Query)
Since May2020 data is within the last 90 days, we could use the API with a URL query to access the measurments for San Francisco in the month of May. However, becasue 2018 and 2019 data is outside of this timeframe, we'll need to use a query tool to query OpenAQ's AWS S3 file storage. 

**URL Parameters & URL for San Francisco**  
City: San Francisco-Oakland-Fremont  
Location: San Francisco  
Parameter: 2.5  
Date: 2020-05-01 to 2020-05-31  
Format: csv  

Clicking the link below downloads a CSV file with the data we specified in the URL.  
https://api.openaq.org/v1/measurements?city=San%20Francisco-Oakland-Fremont&location=San%20Francisco&parameter=pm25&date_from=2020-05-01&date_to=2020-05-31&format=csv  

### OpenAQ AWS S3 Retention (Athena)
Additionally, we can query the entirety of the OpenAQ S3 "bucket". As per the instructions, we must:
1. Create an AWS account
2. Change to the optimal server **US East (N. Virginia)**
3. Create an S3 bucket and folder for our personal data storage.
4. Create a table for the OpenAQ date  
```python
CREATE EXTERNAL TABLE `openaq`(
  `date` struct<utc:string,local:string> COMMENT 'from deserializer', 
  `parameter` string COMMENT 'from deserializer', 
  `location` string COMMENT 'from deserializer', 
  `value` float COMMENT 'from deserializer', 
  `unit` string COMMENT 'from deserializer', 
  `city` string COMMENT 'from deserializer', 
  `attribution` array<struct<name:string,url:string>> COMMENT 'from deserializer', 
  `averagingperiod` struct<unit:string,value:float> COMMENT 'from deserializer', 
  `coordinates` struct<latitude:float,longitude:float> COMMENT 'from deserializer', 
  `country` string COMMENT 'from deserializer', 
  `sourcename` string COMMENT 'from deserializer', 
  `sourcetype` string COMMENT 'from deserializer', 
  `mobile` string COMMENT 'from deserializer')
ROW FORMAT SERDE 
  'org.openx.data.jsonserde.JsonSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://openaq-fetches/realtime-gzipped'
TBLPROPERTIES (
  'transient_lastDdlTime'='1518373755')
```
5. Run a SQL query to retreive data. (2018,2019,2020)  
```python
SELECT *
FROM openaq
WHERE date.utc
    BETWEEN '2018-05-01'
        AND '2018-05-31'
        AND parameter = 'pm25'
        AND value >= 0
        AND location = 'San Francisco'
```

### EPA AirNow API
~ will do later ~

## Load & Explore Data with Python
**Dependencies**

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Import files with read.csv()**

In [2]:
df1 = pd.read_csv("SanFrancisco-May2020-OpenAQ_API.csv")
df2 = pd.read_csv("SanFrancisco-May2020-Athena_Query.csv")

Comparing the number of rows

In [3]:
f"API dataframe has {len(df1)} rows."
f"Query dataframe has {len(df2)} rows."

'API dataframe has 565 rows.'

'Query dataframe has 504 rows.'

Comparing the "head" of both dataframes

In [4]:
df1.head()
df2.head()

,location,city,country,utc,local,parameter,value,unit,latitude,longitude,attribution
0,San Francisco,San Francisco-Oakland-Fremont,US,2020-05-30T01:00:00.000Z,2020-05-29T17:00:00-08:00,pm25,2,µg/m³,37.7658,-122.3978,"[{""name"":""US EPA AirNow"",""url"":""http://www.air..."
1,San Francisco,San Francisco-Oakland-Fremont,US,2020-05-30T00:00:00.000Z,2020-05-29T16:00:00-08:00,pm25,3,µg/m³,37.7658,-122.3978,"[{""name"":""US EPA AirNow"",""url"":""http://www.air..."
2,San Francisco,San Francisco-Oakland-Fremont,US,2020-05-29T23:00:00.000Z,2020-05-29T15:00:00-08:00,pm25,3,µg/m³,37.7658,-122.3978,"[{""name"":""US EPA AirNow"",""url"":""http://www.air..."
3,San Francisco,San Francisco-Oakland-Fremont,US,2020-05-29T22:00:00.000Z,2020-05-29T14:00:00-08:00,pm25,4,µg/m³,37.7658,-122.3978,"[{""name"":""US EPA AirNow"",""url"":""http://www.air..."
4,San Francisco,San Francisco-Oakland-Fremont,US,2020-05-29T21:00:00.000Z,2020-05-29T13:00:00-08:00,pm25,3,µg/m³,37.7658,-122.3978,"[{""name"":""US EPA AirNow"",""url"":""http://www.air..."


,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
0,"{utc=2020-05-02T03:00:00.000Z, local=2020-05-0...",pm25,San Francisco,3,µg/m³,San Francisco-Oakland-Fremont,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=37.7658, longitude=-122.3978}",US,AirNow,government,False
1,"{utc=2020-05-04T04:00:00.000Z, local=2020-05-0...",pm25,San Francisco,5,µg/m³,San Francisco-Oakland-Fremont,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=37.7658, longitude=-122.3978}",US,AirNow,government,False
2,"{utc=2020-05-02T00:00:00.000Z, local=2020-05-0...",pm25,San Francisco,4,µg/m³,San Francisco-Oakland-Fremont,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=37.7658, longitude=-122.3978}",US,AirNow,government,False
3,"{utc=2020-05-03T04:00:00.000Z, local=2020-05-0...",pm25,San Francisco,2,µg/m³,San Francisco-Oakland-Fremont,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=37.7658, longitude=-122.3978}",US,AirNow,government,False
4,"{utc=2020-05-06T06:00:00.000Z, local=2020-05-0...",pm25,San Francisco,10,µg/m³,San Francisco-Oakland-Fremont,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=37.7658, longitude=-122.3978}",US,AirNow,government,False


## Cleaning the Data

~ will do later ~